# Day 5: If you give a seed a fertilizer

I've reached the gardener, who tells me there's no water because there's no sand to filter it with. He directs me to the ferry, but there's also an urgent food production problem. Here's a sample of the almanac:

In [1]:
sample_input = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

Each line within a map contains three numbers: the destination range start, the source range start, and the range length. So seed numbers 50 and 51 correspond to soil numbers 98 and 99. Seeds 52 to 99 correspond to soil 50 to 97. Seeds that aren't mapped (0 to 49) correspond to the same soil number.

Working through the example:
-   Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
-   Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
-   Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
-   Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.

## Part 1

The gardener wants to start with the closest seed location, which in this case would be 35. Naturally, the puzzle input is more complicated. What is the closest seed location in the puzzle input?

In [2]:
def build_almanac(a_str):
    almanac = {}
    almanac['seed_types'] = a_str[a_str.find('seeds:')+6:a_str.find('seed-to-soil map:')].strip().split()
    almanac['seed_soil'] = a_str[a_str.find('seed-to-soil map:')+17:a_str.find('soil-to-fertilizer map:')].strip().split('\n')
    almanac['soil_fertilizer'] = a_str[a_str.find('soil-to-fertilizer map:')+23:a_str.find('fertilizer-to-water map:')].strip().split('\n')
    almanac['fertilizer_water'] = a_str[a_str.find('fertilizer-to-water map:')+24:a_str.find('water-to-light map:')].strip().split('\n')
    almanac['water_light'] = a_str[a_str.find('water-to-light map:')+19:a_str.find('light-to-temperature map:')].strip().split('\n')
    almanac['light_temperature'] = a_str[a_str.find('light-to-temperature map:')+25:a_str.find('temperature-to-humidity map:')].strip().split('\n')
    almanac['temperature_humidity'] = a_str[a_str.find('temperature-to-humidity map:')+28:a_str.find('humidity-to-location map:')].strip().split('\n')
    almanac['humidity_location'] = a_str[a_str.find('humidity-to-location map:')+25:].strip().split('\n')
    return almanac

almanac = build_almanac(sample_input)
almanac

{'seed_types': ['79', '14', '55', '13'],
 'seed_soil': ['50 98 2', '52 50 48'],
 'soil_fertilizer': ['0 15 37', '37 52 2', '39 0 15'],
 'fertilizer_water': ['49 53 8', '0 11 42', '42 0 7', '57 7 4'],
 'water_light': ['88 18 7', '18 25 70'],
 'light_temperature': ['45 77 23', '81 45 19', '68 64 13'],
 'temperature_humidity': ['0 69 1', '1 0 69'],
 'humidity_location': ['60 56 37', '56 93 4']}

In [3]:
def get_destination(source_item, map):
    for range in map:
        dst_start = int(range.split()[0])
        src_start = int(range.split()[1])
        length = int(range.split()[2])
        if (source_item >= src_start) and (source_item <= src_start + length-1):
            return dst_start + source_item - src_start
    return source_item


get_destination(79, almanac['seed_soil'])

81

In [4]:
def trace_seed_location(seed_number, almanac):
    soil = get_destination(seed_number, almanac['seed_soil'])
    fertilizer = get_destination(soil, almanac['soil_fertilizer'])
    water = get_destination(fertilizer, almanac['fertilizer_water'])
    light = get_destination(water, almanac['water_light'])
    temperature = get_destination(light, almanac['light_temperature'])
    humidity = get_destination(temperature, almanac['temperature_humidity'])
    location = get_destination(humidity, almanac['humidity_location'])
    #print(f'Seed {seed_number}, soil {soil}, fertilizer {fertilizer}, water {water}, light {light},',
    #      f'temperature {temperature}, humidity {humidity}, location {location}.')
    return location


trace_seed_location(13, almanac)

35

In [5]:
def find_closest_location(almanac):
    seed_types = almanac['seed_types']
    minimum_distance = 1e99
    for seed in seed_types:
        seed_number = int(seed)
        location = trace_seed_location(seed_number, almanac)
        minimum_distance = min(minimum_distance, location)
    return minimum_distance


find_closest_location(almanac)

35

In [6]:
with open('data/day05_in.txt', 'r') as f:
    puzzle_input = f.read()

In [7]:
almanac = build_almanac(puzzle_input)
find_closest_location(almanac)

289863851

289863851 is correct for my puzzle input.

# Part 2

Oops, the seeds line actually describes ranges of seed numbers. Each pair of numbers includes the start and length of a range.
We're still looking for the lowest location number, but considering a lot more seed numbers. For the example almanac, this value is 46.

In [8]:
almanac = build_almanac(sample_input)
seeds = almanac['seed_types']

while len(seeds) > 0:
    start = int(seeds.pop(0))
    length = int(seeds.pop(0))
    print(f'{start}..{start + length-1}')

79..92
55..67


In [9]:
def find_closest_location(almanac):
    seeds = almanac['seed_types']
    minimum_distance = 1e99
    while len(seeds) > 0:
        start = int(seeds.pop(0))
        length = int(seeds.pop(0))
        for seed in range(start, start+length):
            seed_number = int(seed)
            location = trace_seed_location(seed_number, almanac)
            minimum_distance = min(minimum_distance, location)
    return minimum_distance


In [10]:
find_closest_location(build_almanac(sample_input))

46

That doesn't work! We need another approach for these ranges whose lengths are in the trillions. 

First, let's tidy up our virtual almanac.

In [11]:
def format_seeds(seeds):
    '''Convert the seed list from a list of strings into a list of integers'''
    new_seeds = []
    while len(seeds) > 0:
        new_seeds.append(int(seeds.pop(0)))
    return new_seeds

group_seeds(['79', '14', '55', '13'])

[79, 14, 55, 13]

In [12]:
def format_ranges(map):
    '''Convert each range in a map from a string to a list of integers.'''
    new_map = []
    while len(map) > 0:
        new_range = []
        range = map.pop(0).split()
        while len(range) > 0:
            new_range.append(int(range.pop(0)))
        new_map.append(new_range)
    return new_map

format_ranges(['50 98 2', '52 50 48'])

[[50, 98, 2], [52, 50, 48]]

In [13]:
def build_almanac(a_str):
    '''Parse the input into a dictionary of maps.'''
    almanac = {}
    almanac['seeds'] = format_seeds(a_str[a_str.find('seeds:')+6:a_str.find('seed-to-soil map:')].strip().split())
    almanac['seed_soil'] = format_ranges(a_str[a_str.find('seed-to-soil map:')+17:
                                         a_str.find('soil-to-fertilizer map:')].strip().split('\n'))
    almanac['soil_fertilizer'] = format_ranges(a_str[a_str.find('soil-to-fertilizer map:')+23:
                                               a_str.find('fertilizer-to-water map:')].strip().split('\n'))
    almanac['fertilizer_water'] = format_ranges(a_str[a_str.find('fertilizer-to-water map:')+24:
                                                a_str.find('water-to-light map:')].strip().split('\n'))
    almanac['water_light'] = format_ranges(a_str[a_str.find('water-to-light map:')+19:
                                           a_str.find('light-to-temperature map:')].strip().split('\n'))
    almanac['light_temperature'] = format_ranges(a_str[a_str.find('light-to-temperature map:')+25:
                                                 a_str.find('temperature-to-humidity map:')].strip().split('\n'))
    almanac['temperature_humidity'] = format_ranges(a_str[a_str.find('temperature-to-humidity map:')+28:
                                                    a_str.find('humidity-to-location map:')].strip().split('\n'))
    almanac['humidity_location'] = format_ranges(a_str[a_str.find('humidity-to-location map:')+25:].strip().split('\n'))
    return almanac

almanac = build_almanac(sample_input)
almanac

{'seeds': [79, 14, 55, 13],
 'seed_soil': [[50, 98, 2], [52, 50, 48]],
 'soil_fertilizer': [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 'fertilizer_water': [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 'water_light': [[88, 18, 7], [18, 25, 70]],
 'light_temperature': [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 'temperature_humidity': [[0, 69, 1], [1, 0, 69]],
 'humidity_location': [[60, 56, 37], [56, 93, 4]]}

In [14]:
def transform_single(source, map):
    '''Use a single map range to transform a single source range to one or more destination ranges'''
    source_start = source[0]
    source_length = source[1]
    map_dst_start = map[0]
    map_src_start = map[1]
    map_length = map[2]
    dest = []
    unchanged = []

    if source_start < map_src_start: 
        # source begins before the map
        if source_start + source_length <= map_src_start: 
            # source is completely before the map
            unchanged.extend(source)
        else:
            # source is partially before the map
            unchanged.extend([source_start, map_src_start - source_start])
            if source_start + source_length <= map_src_start + map_length:
                # remainder of source is within the map, apply offset
                dest.extend([map_dst_start, source_start + source_length - map_src_start])
            else: # source extends beyond the map, apply offset to the portion within the map
                dest.extend([map_dst_start, map_length])
                unchanged.extend([map_src_start + map_length, source_start + source_length - (map_src_start + map_length)])

    elif source_start < map_src_start + map_length:
        # source begins within the map
        if source_start + source_length <= map_src_start + map_length:
            # source is completely within the map, apply offset.
            dest.extend([source_start + (map_dst_start - map_src_start), source_length])
        else: # source extends beyond the map, apply offset to the portion within the map
            dest.extend([source_start + (map_dst_start - map_src_start), map_src_start + map_length - source_start])
            unchanged.extend([map_src_start + map_length, source_start + source_length - (map_src_start + map_length)])

    else: # source is completely beyond the map
        unchanged.extend(source)
        
    return dest, unchanged
        

transform_single([1,20], [50, 10, 3]) # transform 10-12 to 50-52

([50, 3], [1, 9, 13, 8])

In [27]:
def transform_multimap(source, multimap):
    '''Loop a single source range through a multi-range map '''
    dest = []
    ranges = source
    for map in multimap:
        next = []
        while len(ranges) > 0:
            if type(ranges[0]) == list:
                range = ranges.pop(0)
            else:
                range = [ranges.pop(0), ranges.pop(0)]
            changed, unchanged = transform_single(range, map)
            dest.extend(changed)
            next.extend(unchanged)
        ranges = next
    dest.extend(next)
    return dest

transform_multimap([55,5], [[50, 10, 3], [52, 50, 48]])


[57, 5]

In [28]:
def transform_multirange(multisource, multimap):
    dest = []
    while len(multisource) > 0:
        if type(multisource[0]) == list:
            source = multisource[0]
        else:
            source = [multisource.pop(0), multisource.pop(0)]
        dest.extend(transform_multimap(source, multimap))
    return dest

transform_multirange([79, 14, 55, 13], almanac['seed_soil'])

[1626068843, 14, 1626068819, 13]

In [29]:
transform_multirange([81, 14, 57, 13], almanac['soil_fertilizer'])

[360004398, 14, 360004374, 13]

In [30]:
def solve_part2(input):
    almanac = build_almanac(input)
    soil = transform_multirange(almanac['seeds'], almanac['seed_soil'])
    fertilizer = transform_multirange(soil, almanac['soil_fertilizer'])
    water = transform_multirange(fertilizer, almanac['fertilizer_water'])
    light = transform_multirange(water, almanac['water_light'])
    temperature = transform_multirange(light, almanac['light_temperature'])
    humidity = transform_multirange(temperature, almanac['temperature_humidity'])
    location = transform_multirange(humidity, almanac['humidity_location'])
    
    range_beginnings = []
    for startposition in range(0, len(location), 2):
        range_beginnings.append(location[startposition])
    
    return min(range_beginnings)

solve_part2(sample_input)

46

In [31]:
solve_part2(puzzle_input)

60568880

60568880 is correct!

I'm sure there's a more elegant way to do that, though.